# CSCI405: Artificial Intelligence
## Module 2
## Assignment 4: Custom Transformer and Transformation Pipeline

#### Name: Jermaine Presbery

Begin by writing your name above.

Your task in this assignment is to create a custom transformation pipeline that takes in raw data and returns fully prepared, clean data that is ready for model training.  However, we will not actually train any models in this assignment.  This pipeline will employ an imputer class, a user-defined transformer class, and a data-normalization class.

Please note that the order of features in the final feature matrix must be correct.  See the below figure that illustrates the input and output of the transformation pipeline.  The positions of features $x_1$ and $x_2$ do not change - they remain in the first and second columns, respectvely, both before and after the transformation pipeline.  In the transformed dataset, the $x_5$ feature is next, and is followed by the newly computed feature $x_6$.  Finally, the last two columns are the remaining one-hot vectors obtained from encoding the categorical feature $x_3$.

<img src="DataTransformation.png " width ="500" />

# Import Data

Import data from the file called `CustomTransformerData.csv`.

In [1]:
# Common imports
import numpy as np
import pandas as pd

# importation of data
fileName = "CustomTransformerData.csv"
df = pd.read_csv(fileName)

# Print out first 5 rows of data
df.head()

,x1,x2,x3,x4,x5
0,1.5,2.354153,COLD,593,0.750000
1,2.5,3.314048,WARM,340,2.083333
2,3.5,4.021604,COLD,551,4.083333
3,4.5,NaN,COLD,2368,6.750000
4,5.5,5.847601,WARM,2636,10.083333


# Create Custom Transformer

Create a custom transformer, just as we did in the lecture video entitled "Custom Transformers", that performs two computations: 

1. Adds an attribute to the end of the data (i.e. new last column) that is equal to $\frac{x_1^3}{x_5}$ for each observation

2. Drops the entire $x_4$ feature column.  (See further instructions below.)

You must name your custom transformer class `Assignment4Transformer`.  Your class should include a parameter with a default value of `True` that deletes the $x_4$ feature column when its value is `True`, but preserves the $x_4$ feature column when its value is `False`.

NOTE: You must handle the numeric and categorical features separately.  Accordingly, you will not pass the $x_3$ feature column through this custom transformer.  This means your calculations should reflect the absence of the $x_3$ feature column when indexing data structures.

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin  #Importation of classes

#Creating a class 
class Assignment4Transformer(BaseEstimator, TransformerMixin):
    def __init__ (self, drop_x4 = True, y = None):
        self.drop_x4 = drop_x4 
    def fit(self, df, y = None):
        return self
    def transform(self, df, y = None): 
        new_Col = df[:,0] **3 / df[:,3]
        if self.drop_x4:
            df = np.delete(df, 2, axis = 1)
        return np.c_[df, new_Col]

# Create Transformation Pipeline for Numerical Features

Create a custom transformation pipeline for numeric data only called `num_pipeline` that:

1. Applies the `SimpleImputer` class to the data, where the strategy is set to `mean`.

2. Applies the custom `Assignment4Transformer` class to the data.

3. Applies the `StandardScaler` class to the data.

In [3]:
#importation of classes






\
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

#Creating a num_pipeline
num_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy = "mean")),
            ('attribs_adder', Assignment4Transformer()),
            ('std_scaler', StandardScaler())])

# Create Numeric and Categorical DataFrames

Create two new data frames.  Create one DataFrame called `data_num` that holds the numeric features.  Create another DataFrame called `data_cat` that holds the categorical features.

In [4]:
#Creating Nmeric and Categorical DataFrames
data_num = df.drop('x3', axis = 1)
data_cat = df.drop(['x1', 'x2', 'x3', 'x4', 'x5'], axis = 1)

# Quick Testing

The full pipeline will be implemented with a `ColumnTransformer` class.  However, to be sure that our numeric pipeline is working properly, lets invoke the `fit_transform()` method of the `num_pipeline` object.  Then, take a look at the transformed data to be sure all is well.

### Run Pipeline and Create Transformed Numeric Data

In [5]:
#Running num_pipeline
num_Transformed = num_pipeline.fit_transform(data_num)
num_Transformed

array([[-1.63835604, -1.72914963, -1.19507691, -1.59050349],
       [-1.44560827, -1.52555901, -1.15738431, -1.39982426],
       [-1.2528605 , -1.37548847, -1.10084543, -1.20914502],
       [-1.06011273,  0.        , -1.02546024, -1.01846579],
       [-0.86736496, -0.9882004 , -0.93122876, -0.82778656],
       [-0.67461719, -0.69501705, -0.81815098, -0.63710732],
       [-0.48186942, -0.53226557, -0.68622691, -0.44642809],
       [-0.28912165, -0.27633017, -0.53545654, -0.25574886],
       [-0.09637388, -0.03636359,  0.        , -0.6099295 ],
       [ 0.09637388,  0.128392  , -0.17737691,  0.12560961],
       [ 0.28912165,  0.26571811,  0.02993235,  0.31628884],
       [ 0.48186942,  0.4501331 ,  0.25608791,  0.50696808],
       [ 0.67461719,  0.75841437,  0.50108976,  0.69764731],
       [ 0.86736496,  0.88038895,  0.76493791,  0.88832654],
       [ 1.06011273,  0.        ,  1.04763235,  1.07900578],
       [ 1.2528605 ,  1.41623801,  1.34917309,  1.26968501],
       [ 1.44560827,  1.

### One-Hot Encode Categorical Features

Similarly, you will employ a `OneHotEncoder` class in the `ColumnTransformer` below to construct the final full pipeline.  However, let's instantiate an object of the `OneHotEncoder` class called `cat_encoder` that has the `drop` parameter set to `first`.  Next, call the `fit_transform()` method and pass it your categorical data.  Take a look at the transformed one-hot vectors to be sure all is well.

In [6]:
from sklearn.preprocessing import OneHotEncoder #Importing onehotencoder
cat_encoder = OneHotEncoder(drop = "first")   #

data_catTransformed = cat_encoder.fit_transform(data_cat) #OneHot encoder presented 
data_catTransformed.toarray() #Data converted to array 

array([], shape=(18, 0), dtype=float64)

# Put it All Together with a Column Transformer

Now, we are finally ready to construct the full transformation pipeline called `full_pipeline` that will transform our raw data into clean, ready-to-train data.  Construct this ColumnTransformer below, then call the `fit_transform()` method to obtain the final, clean data.  Save this output data into a variable called `data_trans`.

In [7]:
from sklearn.compose import ColumnTransformer # Importing ColumnTranformer 

# ColumnTransformer below
full_pipeline = ColumnTransformer([("num_transformed", num_pipeline, [0, 1, 3, 4]),
                                    ("cat_transformed", cat_encoder, [2])])

data_trans = full_pipeline.fit_transform(df)
data_trans

array([[-1.63835604, -1.72914963, -1.19507691, -1.59050349,  0.        ,
         0.        ],
       [-1.44560827, -1.52555901, -1.15738431, -1.39982426,  0.        ,
         1.        ],
       [-1.2528605 , -1.37548847, -1.10084543, -1.20914502,  0.        ,
         0.        ],
       [-1.06011273,  0.        , -1.02546024, -1.01846579,  0.        ,
         0.        ],
       [-0.86736496, -0.9882004 , -0.93122876, -0.82778656,  0.        ,
         1.        ],
       [-0.67461719, -0.69501705, -0.81815098, -0.63710732,  0.        ,
         1.        ],
       [-0.48186942, -0.53226557, -0.68622691, -0.44642809,  1.        ,
         0.        ],
       [-0.28912165, -0.27633017, -0.53545654, -0.25574886,  0.        ,
         0.        ],
       [-0.09637388, -0.03636359,  0.        , -0.6099295 ,  0.        ,
         1.        ],
       [ 0.09637388,  0.128392  , -0.17737691,  0.12560961,  1.        ,
         0.        ],
       [ 0.28912165,  0.26571811,  0.02993235,  0.

# Prepare for Grading

Prepare your `data_trans` NumPy array for grading by using the NumPy [around()](https://numpy.org/doc/stable/reference/generated/numpy.around.html) function to round all the values to 2 decimal places - this will return a NumPy array.

Please note the final order of the features in your final numpy array, which is given at the top of this document.

___You MUST print your final answer, which is the NumPy array discussed above, using the `print()` function!  This MUST be the only `print()` statement in the entire notebook!  Do not print anything else using the print() function in this notebook!___

In [8]:
print(np.around(data_trans,decimals=2))

[[-1.64 -1.73 -1.2  -1.59  0.    0.  ]
 [-1.45 -1.53 -1.16 -1.4   0.    1.  ]
 [-1.25 -1.38 -1.1  -1.21  0.    0.  ]
 [-1.06  0.   -1.03 -1.02  0.    0.  ]
 [-0.87 -0.99 -0.93 -0.83  0.    1.  ]
 [-0.67 -0.7  -0.82 -0.64  0.    1.  ]
 [-0.48 -0.53 -0.69 -0.45  1.    0.  ]
 [-0.29 -0.28 -0.54 -0.26  0.    0.  ]
 [-0.1  -0.04  0.   -0.61  0.    1.  ]
 [ 0.1   0.13 -0.18  0.13  1.    0.  ]
 [ 0.29  0.27  0.03  0.32  0.    1.  ]
 [ 0.48  0.45  0.26  0.51  0.    1.  ]
 [ 0.67  0.76  0.5   0.7   0.    0.  ]
 [ 0.87  0.88  0.76  0.89  1.    0.  ]
 [ 1.06  0.    1.05  1.08  1.    0.  ]
 [ 1.25  1.42  1.35  1.27  0.    1.  ]
 [ 1.45  1.55  1.67  1.46  1.    0.  ]
 [ 1.64  1.71  2.01  1.65  1.    0.  ]]
